In [1]:
# load and standardize data

import mkgu

neural_data = mkgu.get_assembly(name="dicarlo.Majaj2015")
neural_data.load()
neural_data = neural_data.sel(variation=6).multi_groupby(['category_name', 'object_name', 'image_id']) \
    .mean(dim='presentation').squeeze('time_bin').T
# Mostly, we compare neural data with computational models, 
# for deep neural networks see https://github.com/mschrimpf/brain-score-models.
# This repository is agnostic of the comparison system, 
# To show-case the functionality, we are going to compare different regions.
v4_data = neural_data.sel(region='V4')
it_data = neural_data.sel(region='IT')

### Neural fit

In [4]:
# To compare two systems, we instantiate the metric and call it on the source and target assembly.
from mkgu.metrics.neural_fit import NeuralFit

neural_fit_metric = NeuralFit()
score = neural_fit_metric(v4_data, it_data)
# This gives us a score, containing the raw `values` that went into the score. 
# For instance, there is one value per cross-validation split.
# Since we often care only about a value and error bars, 
# it also contains an `aggregation` with a center and error (typically mean and s.e.m.).
print("full score: ", score, "\n")
# Since the score is potentially multi-dimensional, it is also structured as an xarray DataArray.
# We can select the center as follows.
print("center:", score.aggregation.sel(aggregation='center'))


/home/martin/miniconda3/envs/neurality/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


full score:  NeuroidMedianScore(values=<xarray.DataAssembly (split: 10)>
array([0.563077, 0.550949, 0.559204, 0.523328, 0.556234, 0.528516, 0.528036,
       0.547969, 0.543635, 0.54202 ])
Coordinates:
  * split    (split) int64 0 1 2 3 4 5 6 7 8 9,aggregation=<xarray.DataAssembly (aggregation: 2)>
array([0.544297, 0.004164])
Coordinates:
  * aggregation  (aggregation) object 'center' 'error') 

center: <xarray.DataAssembly ()>
array(0.544297)
Coordinates:
    aggregation  <U6 'center'


### RDM

In [5]:
# We can easily swap out the specific metric and use e.g. RDMs.
# To compare two systems, we instantiate the metric and call it on the source and target assembly.
from mkgu.metrics.rdm import RDMMetric

rdm = RDMMetric()
score = rdm(v4_data, it_data)
print("center:", score.aggregation.sel(aggregation='center'))
# Note how the score is much lower with RDMs due to missing re-mapping.

/home/martin/miniconda3/envs/neurality/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


center: <xarray.DataAssembly ()>
array(0.286585)
Coordinates:
    aggregation  <U6 'center'
